<a href="https://colab.research.google.com/github/The237/DeepLearningCourses/blob/main/03_01_Deep_Learning_pour_la_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [3]:
data = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [11]:
training_labels.reshape((training_images.shape[0],1))


array([[9],
       [0],
       [0],
       ...,
       [3],
       [0],
       [5]], dtype=uint8)

In [12]:
training_labels.shape

(60000,)